In [ ]:
# -*- coding:utf-8 -*-
## crawler for 和欣客運
import requests
from bs4 import BeautifulSoup as bs
import time

In [ ]:
url = "https://www.ebus.com.tw/online_trans/Web/Add_Step1.asp" ## 網路訂／購票須知 => 按 『我同意』[post]

url2 = "https://www.ebus.com.tw/online_trans/Web/Add_Step2.asp" ## 網路訂／購票 => 頁面會導到 addOrder.aspx
url3 = "https://www.ebus.com.tw/NetOrder/payOrder/addOrder.aspx" ## 輸入起訖站跟姓名、身分證、電話... 頁面.
url3_checkimg = "https://www.ebus.com.tw/NetOrder/CheckImageCode.aspx" ## 有驗證圖片數字需要由回傳 cookies解析[response.cookies明碼]

In [ ]:
headers1={
    "Host":"www.ebus.com.tw",
    "Referer":"http://www.ebus.com.tw/ebus/top.html",
    "Connection":"keep-alive",
    "TE":"Trailers",
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:74.0) Gecko/20100101 Firefox/74.0",
}

payload1={
    "Agreement":"%A7%DA%A6P%B7N"
}

headers2={
    "Host":"www.ebus.com.tw",
    "Connection":"keep-alive",
    "TE":"Trailers",
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:74.0) Gecko/20100101 Firefox/74.0",
    "Referer":"https://www.ebus.com.tw/online_trans/Web/Add_Step1.asp",    
}


In [ ]:
### ===== function 預先定義區塊 =====
def parsing_hidden_params(soup):
    __VIEWSTATE = soup.select("input#__VIEWSTATE")[0]['value']
    __VIEWSTATEGENERATOR = soup.select("input#__VIEWSTATEGENERATOR")[0]['value']
#     __EVENTVALIDATION = soup.select("input#__EVENTVALIDATION")[0]['value']
    __VIEWSTATEENCRYPTED = soup.select("input#__VIEWSTATEENCRYPTED")[0]['value']
    __PREVIOUSPAGE = soup.select("input#__PREVIOUSPAGE")[0]['value']
    
    
    output = {
        "__VIEWSTATE":__VIEWSTATE,
        "__VIEWSTATEGENERATOR":__VIEWSTATEGENERATOR,
#         "__EVENTVALIDATION":__EVENTVALIDATION
        "__VIEWSTATEENCRYPTED":__VIEWSTATEENCRYPTED,
        "__PREVIOUSPAGE":__PREVIOUSPAGE,
    }
    return output

def parsing_checkcode(responsed):
    return responsed.cookies['CheckCode']

def get_personal_data(filename):
    filedata = []
    try:
        with open(filename,"r",encoding="utf8") as f6:
            filedata = f6.readlines()
    except:
        print("get_personal_data() didn't find a {} in the same directory! \n".format(filename))
        print("A new {} file will be created! Please try to fill up your data with another editor!".format(filename))
        with open(filename,"w",encoding="utf8") as f5:
            f5.write("id={}\n".format(txtCId))
            f5.write("name={}\n".format(txtCName))
            f5.write("tel={}\n".format(txtCTel))
        return ["","",""]
    else:
        return filedata


In [ ]:
# === main() ===


filedata = get_personal_data("personal.txt")
txtCId=filedata[0].replace("\n","").replace("id=","")
txtCName=filedata[1].replace("\n","").replace("name=","")
txtCTel=filedata[2].replace("\n","").replace("tel=","")


# === connection process ===
session1=requests.Session()
session1.headers = headers1

response1 = session1.get(url)
soup1 = bs(response1.text,'lxml')
print(soup1.select("input"))

time.sleep(2)

print("="*100,"\n=\n=\n= [python] 以下為 {} POST 後結果: \n=\n".format(url2),"="*100)
response2 = session1.post(url2,headers=headers1,data=payload1)
soup2 = bs(response2.text,'lxml')
print(soup2.select("title"))

p = parsing_hidden_params(soup2)
# print(p)
time.sleep(1)

print("="*100,"\n=\n=\n= [python] 以下為 {} GET 後結果: \n=\n".format(url3_checkimg),"="*100)
## 再送一次 get到 url3_checkimg取得驗證碼資料:
response3 = session1.get(url3_checkimg)
CheckCode = parsing_checkcode(response3)


### 準備要送出的參數: 
__EVENTTARGET=""
__EVENTARGUMENT=""
tkData=""
Seat0Number=""
Seat1Number=""
defeStationId=""
SelectAddr="高雄市建國二路255號　Tel：(07)236-0209"
listGsStationId="E1*A1,A2,A3,A4,B1,B3,B4,B9,B5,C1,DA*高雄建國站,高雄中正站,高雄九如站,高雄楠梓站,臺南轉運站,台南鹽行站,台南新市站,台南六甲頂,麻豆站,新營轉運站,台中朝轉站"
listGeStationId="B1"
listGDate="2020/04/16"
listGTime="18:00,21:00"
txtChkCode=CheckCode
butgBusTimeList="下一步"

order_bus_time_payload={
    "__EVENTTARGET":"",
    "__EVENTARGUMENT":"",
    "tkData":"",
    "Seat0Number":"",
    "Seat1Number":"",
    "defeStationId":"",
    "SelectAddr":SelectAddr,
    "txtCName":txtCName,
    "txtCId":txtCId,
    "txtCTel":txtCTel,
    "listGsStationId":listGsStationId,
    "listGeStationId":listGeStationId,
    "listGDate":listGDate,
    "listGTime":listGTime,
    "txtChkCode":txtChkCode,
    "butgBusTimeList":butgBusTimeList,
}
order_bus_time_payload["__VIEWSTATE"]=p["__VIEWSTATE"]
order_bus_time_payload["__VIEWSTATEGENERATOR"]=p["__VIEWSTATEGENERATOR"]
order_bus_time_payload["__VIEWSTATEENCRYPTED"]=p["__VIEWSTATEENCRYPTED"]
order_bus_time_payload["__PREVIOUSPAGE"]=p["__PREVIOUSPAGE"]

time.sleep(1)
print("="*100,"\n=\n=\n= [python] 以下為 {} POST 後結果: \n=\n".format(url3),"="*100)
response4 = session1.post(url3,headers=headers2,data=order_bus_time_payload)
soup4 = bs(response4.text,'lxml')
print(soup4.select("table span#label_gTravelData")[0].text)

session1.close()




In [ ]:
# print(soup4)

In [ ]:
### ==== test 準備這邊寫 parsing table ====

# print(soup4.select("table span#label_gTravelData")[0].text)
# print(soup4.select("td[style='letter-spacing:1pt;']")[0].text)

# print(soup4.select("table#GView_GBusTime tr")[2].select("a")[1]['href'])
dict_buslist ={}
column =[]
column = [ele.text for ele in soup4.select("table#GView_GBusTime th")]
print(column)
tdata=[]
lnks=list()
for tr in soup4.select("table#GView_GBusTime tr"):
    
    tdata=[]
#     print(tr.select("a"))
    for td in tr.select("td"):
        print(len(td),"-"*80)
        lnks+=[lnk['href'] for lnk in td.select("a")]
#         print(len(lnks))
        tdata.append(td.text)
    print(lnks)
    print(tdata)
print(lnks[0])
print(tdata[0])
        
#     dict_buslist["bus_level"]=ele.select("td")[1].text
#     dict_buslist["empty_seat_num"]=ele.select("td")[2].text
#     dict_buslist["comments"]=ele.select("td")[3].text
#     dict_buslist["one_way_trip_url"]=ele.select("a")[0]['href']
#     dict_buslist["round_way_trip_url"]=ele.select("a")[1]['href']

# dict_buslist["buslist"]=soup4.select("table#GView_GBusTime tr")[2].select("td")[0].text
# dict_buslist["bus_level"]=soup4.select("table#GView_GBusTime tr")[2].select("td")[1].text
# dict_buslist["empty_seat_num"]=soup4.select("table#GView_GBusTime tr")[2].select("td")[2].text
# dict_buslist["comments"]=soup4.select("table#GView_GBusTime tr")[2].select("td")[3].text
# dict_buslist["one_way_trip_url"]=soup4.select("table#GView_GBusTime tr")[2].select("a")[0]['href']
# dict_buslist["round_way_trip_url"]=soup4.select("table#GView_GBusTime tr")[2].select("a")[1]['href']
# for ele in soup4.select("table#GView_GBusTime tr a"):
#     print(ele['href'])
print(dict_buslist)    
    

In [ ]:
a = soup4.select("table#GView_GBusTime tr")[1:]
tt = list()
for tr in a:
    tt.append([td['href'] for td in tr.find_all("a")])
print(tt
     )

In [ ]:
cookies = requests.utils.dict_from_cookiejar(response1.cookies)
print(cookies)
response1.cookies

In [ ]:
print(requests.utils.dict_from_cookiejar(response1.cookies))
print(requests.utils.dict_from_cookiejar(response2.cookies))
print(requests.utils.dict_from_cookiejar(response3.cookies))

In [ ]:
print(response1.headers)
print(response2.headers)
print(response3.headers)